# Modelo de Vasicek

## Ecuaciones del Modelo

El modelo de Vasicek para la tasa instantánea en la medida histórica es:

$$
\begin{equation}
dr_t=\gamma\left(\overline{r}-r_t\right)dt+\sigma dX_t
\end{equation}
$$

Este modelo también supone que $\lambda$ el *market price of interest rate risk* es constante y por lo tanto, en la medida ajustada por riesgo:

$$
\begin{equation}
dr_t=\gamma^*\left(\overline{r}^*-r_t\right)dt+\sigma dX^*_t
\end{equation}
$$

donde $\gamma^*=\gamma$ y $\overline{r}^*=\overline{r}-\lambda\sigma\gamma^{-1}$.

La solución de esta ecuación es:

$$
\begin{equation}
r_t=r_0 e^{-\gamma^* t}+\overline{r}^*\left(1-e^{-\gamma^* t}\right)+\sigma e^{-\gamma^* t}\int_{0}^{t}e^{\gamma^* s}dX_s
\end{equation}
$$

Y además tenemos que:

$$
\begin{equation}
\mathbb{E}_s\left[r_t\right]=r_0 e^{-\gamma^* \left(t-s\right)}+\overline{r}^*\left(1-e^{-\gamma^* (t-s)}\right) \\
\end{equation}
$$

$$
\begin{equation}
Var_s\left[r_t\right]=\frac{\sigma^2}{2\gamma^*}\left(1-e^{-2\gamma^* (t-s)}\right)
\end{equation}
$$

El modelo de Vasicek admite una fórmula explícita para el valor de un bono cupón cero que vence en $T$:

$$
\begin{equation}
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right] \\
Z\left(r_T,T,T\right)=1
\end{equation}
$$

Donde:

$$
\begin{equation}
B\left(t,T\right)=\frac{1}{\gamma^*}\left\{1-\exp\left[-\gamma^*\left(T-t\right)\right]\right\}
\end{equation}
$$

$$
\begin{equation}
A\left(t,T\right)=\left[B\left(t,T\right)-\left(T-t\right)\right]\left[\overline{r}^*-\frac{\sigma^2}{2{\gamma^*}^2}\right]-\frac{\sigma^2{B\left(t,T\right)}^2}{4\gamma^*}
\end{equation}
$$

### Ejercicio

¿Qué distribución tiene el bono cupón cero $Z$ en el modelo de Vasicek?

Lognormal, porque $r_t$ es normal y $Z=\exp(A-B\cdot r)$ lo que implica que $\log Z$ es normal.

## Simulación del Modelo

Vamos a trabajar primero con parámetros inventados. Haremos una simulación. Luego usando la fórmula para el bono cupón cero estudiaremos el ajuste del modelo a un set de datos reales.

In [8]:
from typing import List, Tuple
import scipy.optimize as opt
import plotly.express as px
import pandas as pd
import numpy as np
import math

In [9]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

In [2]:
# type hints
def func(x: float, y: float) -> float:
    return x + y

Damos de alta los parámetros del modelo (hay que interpretarlos como los parámetros en la medida ajustada por riesgo):

In [63]:
gamma = 1
sigma = .01
r_ = .01
r0 = .01 # tasa corta a día de hoy

Veamos que forma tienen sus trayectorias:

In [35]:
def vasicek_path(
        r0: float,
        gamma: float,
        r_: float,
        sigma: float,
        num_dias: int = 263,
        dias_agno: int = 264,
        pasos_dia: int = 1) -> List[Tuple[float, float]]:
    """
    Retorna un camino de simulación del modelo de Vasicek usando el esquema de Euler.
    
    r(t+dt) = r(t) + gamma * (r_ - r(t)) * dt + sigma * raiz(dt) * eps(t)
    r(t + dt) = gamma * r_ * dt + r(t) *(1 - gamma * dt) + sigma * raiz(dt) * eps(t)
    
    donde eps(t) distibuye N(0,1).
    
    params:

    - r0: tasa inicial (t = 0) de la simulación
    - Parámetros del modelo:
      - gamma: velocidad de reversión
      - r_: tasa de largo plazo
      - sigma: volatilidad
    - num_dias: número de días en la trayectoria, incluyendo el instante t = 0
    - dias_agno: número de días hábiles por año
    - pasos_dia: número de pasos de simulación en 1 día

    return:

    - Un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.
    """
    dt = 1 / (dias_agno * pasos_dia)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0), ]
    gamma_dt = gamma * dt

    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * \
            np.random.normal()  # Discretización de Euler
        result.append((i * dt, r))
    return result

In [64]:
sim = vasicek_path(r0, gamma, r_, sigma, num_dias=1000)
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])
fig = px.line(
    df_sim,
    x='t',
    y='tasa',
    title=f'Simulación Modelo Vasicek. r0={r0:.2%}, gamma={gamma}, r_={r_:.2%}, sigma={sigma:.2%}'
)
fig.update_layout(yaxis_range=[-.04,.04])
fig.show()

## Estimación de Parámetros

Se utilizará la curva OIS que se construyó en clase (notebook 5):

In [65]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [67]:
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


In [9]:
# zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

In [68]:
fig = px.line(df_curva, x='plazo', y='tasa', title='Curva de Mercado')
fig.show()

Se definen las funciones asociadas a la fórmula para obtener un factor de descuento o precio de un bono cero cupón con el modelo de Vasicek.

In [69]:
def b_vasicek(
        r: float,
        r_: float,
        gamma: float,
        t: float,
        T: float
) -> float:
    """
    Implementa la función B que interviene en el cálculo del valor de un bono cupón
    cero con el modelo de Vasicek.

    Z(t, T) = exp(A(t, T) - B(t, T) * r(t))

    params:

    - r: valor de la tasa corta
    - Parámetros del Modelo:
      - r_: tasa de largo plazo
      - gamma: velocidad de reversión
    - t: tiempo de cálculo
    - T: tiempo de vencimiento del bono (debe ser T >= t)

    return:

    - valor de la función B
    """
    return 1 / gamma * (1 - math.exp(-gamma * (T - t)))


def a_vasicek(
        r: float,
        r_: float,
        gamma: float,
        sigma: float,
        t: float,
        T: float
) -> float:
    """
    Implementa la función A que interviene en el cálculo del valor de un bono cupón
    cero con el modelo de Vasicek.

    Z(t, T) = exp(A(t, T) - B(t, T) * r(t))

    params:

    - r: valor de la tasa corta
    - Parámetros del Modelo:
      - r_: tasa de largo plazo
      - gamma: velocidad de reversión
    - t: tiempo de cálculo
    - T: tiempo de vencimiento del bono (debe ser T >= t)

    return:

    - valor de la función A
    """
    b = b_vasicek(r, r_, gamma, t, T)
    sigma2 = sigma ** 2
    return (b - (T - t)) * (r_ - sigma2 / (2 * gamma ** 2)) - (sigma2 * b ** 2) / (4.0 * gamma)


def zero_vasicek(
    r: float,
    r_: float,
    gamma: float,
    sigma: float,
    t: float,
    T: float
) -> float:
    """
    Implementa la fórmula para el cálculo del valor de un bono cupón cero con el modelo de Vasicek.

    Z(t, T) = exp(A(t, T) - B(t, T) * r(t))

    params:

    - r: valor de la tasa corta
    - Parámetros del Modelo:
      - r_: tasa de largo plazo
      - gamma: velocidad de reversión
    - t: tiempo de cálculo
    - T: tiempo de vencimiento del bono (debe ser T >= t)

    return:

    - valor de la función A
    """
    return math.exp(a_vasicek(r, r_, gamma, sigma, t, T) - b_vasicek(r, r_, gamma, t, T) * r)

In [70]:
# Probar la función
df = zero_vasicek(r0, r_, gamma, sigma, 0, 1.5)
print(f'df: {df:.8%}')

df: 98.51326945%


Queremos estimar los parámetros $\gamma^*$, $\overline{r}^*$ y $\sigma$ minimizando el error cuadrático medio entre los factores de descuento obtenidos con la fórmula del modelo y los factores de descuento de la curva de mercado (en `df_curva` más arriba).

Se define la función objetivo para encontrar los parámetros:

In [71]:
# x = [r_, gamma, sigma] valor inicial
def objective(x: List[float], *args) -> float:
    """
    Calcula el error cuadrático total entre los valores de la curva de mercado y los obtenidos
    con la fórmula del modelo de Vasicek y los parámetros x.
    """
    error = 0.0
    # Supongo que el DataFrame con las tasas es el primer elemento de args
    for row in args[0].itertuples(): 
        # args[1] es la tasa corta de hoy
        error += (row.df - zero_vasicek(args[1], x[0], x[1], x[2], 0, row.plazo / 365)) ** 2
    return error

In [74]:
objective(
    [r_, gamma, sigma], df_curva, r0)

0.01708607452228135

In [75]:
print(r_, gamma, sigma)

0.01 1 0.01


Se encuentran los parámetros usando scipy:

In [81]:
x0 = [.03, 0.5, .01]
r0 = df_curva.iloc[0]['tasa']
bnds = ((0, None), (0, None), (0, None))
result = opt.minimize(objective, x0, args=(df_curva, r0), bounds=bnds, tol=.00000001)
print(result)
optimo = result.x

      fun: 0.003306986381573132
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.94039444e-05, 6.28577052e-07, 5.20417043e-09])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 216
      nit: 38
     njev: 54
   status: 0
  success: True
        x: array([0.01031726, 0.22319265, 0.        ])


In [77]:
print(f'r_: {optimo[0]:.4%}')
print(f'gamma: {optimo[1]:.4f}')
print(f'sigma: {optimo[2]:.4%}')

r_: 1.0317%
gamma: 0.2232
sigma: 0.0000%


Z(0,T) = E(exp(-int(0,T)r(s) x ds))

¿Porqué $\sigma$ da 0?

Comparemos la curva real con la aproximación:

In [82]:
v_curva = []
for row in df_curva.itertuples():
    v_curva.append((
        row.plazo / 365.0,
        row.df,
        zero_vasicek(r0, optimo[0], optimo[1], optimo[2], 0, row.plazo / 365)
    ))
compara_dfs = pd.DataFrame(v_curva, columns=['plazo', 'df_mkt', 'df_vasicek'])
fig = px.line(compara_dfs, x='plazo', y=['df_mkt', 'df_vasicek'], title='Compara Vasicek vs Mercado (plazo en años)')
fig.show()

Hagamos la misma comparación, pero en tasa:

In [83]:
def tasa_comp(df: float, plazo: float) -> float:
    return (1 / df) ** (1 / plazo) - 1

compara_dfs['tasa_vasicek'] = compara_dfs.apply(lambda row: tasa_comp(row.df_vasicek, row.plazo), axis=1)
compara_dfs['tasa_mkt'] = compara_dfs.apply(lambda row: tasa_comp(row.df_mkt, row.plazo), axis=1)

In [84]:
compara_dfs.head()

,plazo,df_mkt,df_vasicek,tasa_vasicek,tasa_mkt
0,0.002740,0.999998,0.999998,0.000814,0.000811
1,0.019178,0.999984,0.999984,0.000832,0.000841
2,0.038356,0.999970,0.999967,0.000852,0.000780
3,0.057534,0.999955,0.999950,0.000872,0.000774
4,0.090411,0.999929,0.999918,0.000907,0.000781


In [19]:
fig = px.line(compara_dfs, x='plazo', y=['tasa_mkt', 'tasa_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()

Veamos como queda la simulación:

In [86]:
sim = vasicek_path(r0, optimo[0], optimo[1], optimo[2])
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])
fig = px.line(
    df_sim,
    x='t',
    y='tasa',
    title=f'Simulación Modelo Vasicek. r0={r0:.2%}, gamma={optimo[1]:.4f}, \
    r_={optimo[0]:.2%}, sigma={optimo[2]:.2%}'
)
fig.show()

In [87]:
# x = [r_, gamma] valor inicial
def objective2(x: List[float], *args) -> float:
    """
    Calcula el error cuadrático total entre los valores de la curva de mercado y los obtenidos
    con la fórmula del modelo de Vasicek y los parámetros x.
    """
    r_ = x[0]
    gamma = x[1]
    
    df_tasas = args[0]
    r0 = args[1]
    sigma0 = args[2]
    
    t0 = 0
    
    error = 0.0
    for row in df_tasas.itertuples():
        # 0 es que estoy calibrando la curva a t = 0
        error += (row.df - zero_vasicek(r0, r_, gamma, sigma0, t0, row.plazo / 365)) ** 2
    return error

In [88]:
x00 = [.03, 1]
sigma0 = .02
bnds = ((0, None), (0, None))
result = opt.minimize(objective2, x00, args=(df_curva, r0, sigma0), bounds=bnds)
print(result)
optimo2 = result.x

      fun: 0.0034946069945208007
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.74033964e-05, -3.93222781e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 219
      nit: 55
     njev: 73
   status: 0
  success: True
        x: array([0.02472949, 0.11358796])


In [90]:
print(f'r_: {optimo2[0]:.4%}')
print(f'gamma: {optimo2[1]:.4f}')
print(f'sigma: {sigma0:.4%}')

r_: 2.4729%
gamma: 0.1136
sigma: 2.0000%


In [91]:
v_curva = []
for row in df_curva.itertuples():
    v_curva.append((
        row.plazo / 365.0,
        row.df,
        zero_vasicek(r0, optimo2[0], optimo2[1], sigma0, 0, row.plazo / 365)
    ))
compara_dfs_2 = pd.DataFrame(v_curva, columns=['plazo', 'df_mkt', 'df_vasicek'])
fig = px.line(compara_dfs_2, x='plazo', y=['df_mkt', 'df_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()

In [93]:
compara_dfs_2['tasa_mkt'] = compara_dfs_2.apply(lambda row: tasa_comp(row.df_mkt, row.plazo), axis=1)
compara_dfs_2['tasa_vasicek'] = compara_dfs_2.apply(lambda row: tasa_comp(row.df_vasicek, row.plazo), axis=1)

In [94]:
fig = px.line(compara_dfs_2, x='plazo', y=['tasa_mkt', 'tasa_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()

In [97]:
sim = vasicek_path(r0, optimo2[0], optimo2[1], sigma0)
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])
fig = px.line(
    df_sim, x='t',
    y='tasa',
    title=f'Simulación Modelo Vasicek. r0={r0:.2%}, gamma={optimo2[1]:.4f}, r_={optimo2[0]:.2%}, sigma={sigma0:.2%}'
)
fig.show()

## Fórmulas para Calls y Puts

Consideremos una opción con el siguiente payoff (una opción call sobre un bono cero cupón):

$$
\begin{equation}
Call\left(r_{T_O},T_O\right)=\max\left[Z\left(r_{T_O},T_O,T_B\right)-K,0\right]
\end{equation}
$$

donde $T_O$ representa la fecha de vencimiento de la opción, $T_B$ la fecha de vencimiento del bono y $r_{T_O}$ es la tasa instantánea al vencimiento de la opción.

Tenemos que, bajo el modelo de Vasicek, el valor de esta call cuando $t=0$ está dado por:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$


$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{\sqrt{S_Z\left(T_O\right)}}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

d1 = (ln(F/K) + 1/2 sigma^2 * T) / (sigma x raiz(T))

INT(0, inf) max(S-K,0) dS 

Como se calcula la INT, recordando que S es lognormal y por lo tanto tiene una densidad que se mete dentro de la INT y se resuelve.

### Ejercicio

¿Qué es $S_Z\left(T_O\right)$?

Es la volatilidad del retorno (logarítimico) del bono cupón cero que vence en $T_B$ desde $t=0$ hasta $t=T_O$.

Análogamente para una opción put sobre un bono cero cupón tenemos que:

$$
\begin{equation}
Put\left(r_{T_O},T_O\right)=\max\left[K-Z\left(r_{T_O},T_O,T_B\right),0\right]
\end{equation}
$$



Y su valor según el modelo de Vasicek está dado por:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

y los mismos valores anteriores para $d_1$ y $d_2$.

## Fórmulas para Caps y Floors

Veamos como podemos utilizar la fórmula para valorizar opciones sobre bonos cupón cero para valorizar caps y floors.

Sea $L\left(T-\Delta,T\right)$ la tasa Libor fijada en $T-\Delta$ y que vence en $T$. El valor presente del payoff de un caplet que cuyo fixing se produce en $T-\Delta$ está dado por ($N$ es el nocional del caplet):

$$
\begin{equation}
PV_{T_\Delta}\left(payoff\right)=\frac{N}{1+L\left(T-\Delta,T\right)\Delta}\Delta\max\left[L\left(T-\Delta,T\right)-L_K,0\right] \\
=\frac{N}{1+L\left(T-\Delta,T\right)\Delta}\max\left[\left(1+L\left(T-\Delta,T\right)\Delta\right)-\left(1+L_K\Delta\right),0\right]
\end{equation} \\
=N\max\left[1-\frac{1+L_K\Delta}{1+L\left(T-\Delta,T\right)\Delta}\right] \\
=N\left(1+L_K\Delta\right)\max\left[\frac{1}{1+L_K\Delta}-\frac{1}{1+L\left(T-\Delta,T\right)\Delta}\right]
$$

Notamos entonces que el caplet es equivalente a una **put** sobre el bono $Z\left(T−\Delta,T\right)$ con strike igual a $\frac{1}{\left(1+L_K\Delta\right)}$ y nocional igual a $N\left(1+L_K\Delta\right)$. Por lo tanto, en el modelo de Vasicek, un caplet de strike $L_K$ , vencimiento $T-\Delta$ y nocional igual a $N$ está dado por:

$$
\begin{equation}
V\left(r_0,0)\right)=MZ\left(r_0,0,T-\Delta\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T\right)}{Z\left(r_0,0,T-\Delta\right)}N\left(-d_1\right)\right]
\end{equation}
$$

$$
\begin{equation}
M=N\left(1+L_{K}\Delta\right) \\
K=\frac{1}{1+L_{K}\Delta}
\end{equation}
$$

$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T\right)}{KZ\left(r_0,0,T-\Delta\right)}\right)+\frac{1}{2}S_Z\left(T-\Delta\right)^2}{\sqrt{S_Z\left(T-\Delta\right)}}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T-\Delta\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T-\Delta\right)=B\left(T-\Delta,T\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left[1-\exp\left(-2\gamma^* \left(T-\Delta\right)\right)\right]}
\end{equation} 
$$